# Diagnosis based on ECG data

- - - 

* Diagnosis using collected ECG data
* Currently total 42 subjects
* 3 classes (DEP, SUI, NOR)

- - -

In [ ]:
# importing required components 
import os
import time
import json
import urllib
import random
import torch
import torchvision
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
from PIL import Image
from pdf2image import convert_from_path
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from efficientnet_pytorch import EfficientNet

## Basic data check

In [ ]:
ecg_df = pd.read_csv('E:/RESEARCH/Datasets/wearable/AI_coded_1.csv', sep=',')

* 1: depression, 2: suicidality, 3: normal

In [ ]:
ecg_df['class'] = ecg_df['class'].astype("category")
ecg_df['sub'] = ecg_df['sub'].astype("category")

In [ ]:
depression = ecg_df[ecg_df['class']==1]
print((depression.index)+1)

In [ ]:
suicidal = ecg_df[ecg_df['class']==2]
print((suicidal.index)+1)

In [ ]:
normal = ecg_df[ecg_df['class']==3]
print((normal.index)+1)

- - -

## Image Data handling

In [ ]:
## Checking file path and names
file_path = "E:/RESEARCH/Datasets/wearable/ECG/test_0420/train/dep/"
file_names = os.listdir(file_path)

In [ ]:
## Changing file names to 1, 2, ...
i = 1
for name in file_names:
    src = os.path.join(file_path, name)
    dst = str(i) + '.png'
    dst = os.path.join(file_path, dst)
    os.rename(src, dst)
    i += 1

In [ ]:
## Converting pdf file into png file
for name in file_names:
    pages = convert_from_path(file_path + name, poppler_path="E:/RESEARCH/Datasets/wearable/ECG/poppler/Library/bin")
    
    for page in pages:
        page.save(file_path + name + '.png', "PNG")

In [ ]:
## Data Crop (deleting patients' information)
# original png size = 2200 x 1700
# leaving the important part only (only the ecg data part)
left = 100
top = 450
right = 2100
bottom = 1300
## 100, 450, 2100, 1300 스케일로 자르면 딱 ecg 30초 전체 부분 나옴.

for name in file_names:
    im = Image.open(file_path + name)
    imc = im.crop((left, top, right, bottom))
    imc.save(file_path+name+'.png')

In [ ]:
## directory setting and get file names for 2 second cycle of ECG
crop_path = "E:/RESEARCH/Datasets/wearable/ECG/test_0420/train_crop/sui/"
crop_names = os.listdir(crop_path)

In [ ]:
# Setting the points for cropped image
left = 1000
top = 0
right =1400
bottom = 250
## 위에서 한번 처리한 이미지에 대해 1000, 0, 1400, 250로 자르면 딱 5~7초 ecg cycle 나옴.


for name in crop_names:
    im = Image.open(crop_path + name)
    imc = im.crop((left, top, right, bottom))
    imc.save(crop_path+name+'.png')

In [ ]:
## Changing file names to 1, 2, ...
i = 1
for name in crop_names:
    src = os.path.join(crop_path, name)
    dst = str(i) + '.png'
    dst = os.path.join(crop_path, dst)
    os.rename(src, dst)
    i += 1

- - -

- - -

## With simple image classification

* Our dataset numbers (dep: 244, nor: 402, sui: 105)

In [ ]:
class Args:
    # arugments
    epochs=50
    bs=16
    lr=0.001
    momentum=0.9
    num_channels=3 
    num_classes=3
    verbose='store_true'
    seed=710674

args = Args()    

np.random.seed(args.seed)
random.seed(args.seed)
torch.manual_seed(args.seed)

In [ ]:
#Setting torch environment

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
    
print('Using PyTorch version:', torch.__version__, ' Device: ', DEVICE)

In [ ]:
# model_res = models.resnet18(num_classes=2, pretrained=True)
model_eff3 = EfficientNet.from_pretrained('efficientnet-b3', num_classes=args.num_classes)
# model_resnet18 = models.resnet18(pretrained=True)
# model_mobnetv2 = models.mobilenet_v2(pretrained=True)

In [ ]:
model_mobnetv2

In [ ]:
## resnet 구조는 마지막 fc layer의 out_features 를 바꿔주면 되고.
model_resnet18.fc = nn.Linear(in_features = 512, out_features = args.num_classes)
# model_mobnetv2.classifier = nn.Linear(in_features = 512, out_features = args.num_classes)

In [ ]:
# model = model_res.to(DEVICE)
model = model_eff3.to(DEVICE)
# model = model_mobnetv2.to(DEVICE)
# model = model_resnet18.to(DEVICE)

In [ ]:
# Data Transformation
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(256),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomVerticalFlip(),
    transforms.ColorJitter(contrast=(0.3, 1), saturation=(0.3, 1)),
    transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456,0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Uploading image data
# ecg_data = datasets.ImageFolder(root = 'E:/RESEARCH/Datasets/wearable/ECG/test_0420/train', transform = data_transforms)
ecg_data = datasets.ImageFolder(root = 'E:/RESEARCH/Datasets/wearable/ECG/test_0420/train_crop', transform = data_transforms)

In [ ]:
train_size = int(0.8 * len(ecg_data))
test_size = len(ecg_data)-train_size
print(train_size)
print(test_size)

In [ ]:
train_dataset, test_dataset = torch.utils.data.random_split(ecg_data, [train_size, test_size])

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.bs, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.bs, shuffle=False, num_workers=4)

In [ ]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(labels)

In [ ]:
# Setting Optimizer and Objective Function

optimizer = torch.optim.Adam(model.parameters(), lr = args.lr)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001, total_steps=50, anneal_strategy='cos')
criterion = nn.CrossEntropyLoss() ## setup the loss function

# print(model)

In [ ]:
# Function for checking model performance during CNN model

def train(model, train_loader, optimizer, log_interval):
    model.train()
    print(optimizer.param_groups[0]['lr'])
    
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))

    scheduler.step() #for learning rate scheduler

In [ ]:
# Function for checking model performance during the learning process

def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    validation =[]

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
            
    
    test_loss /= (len(test_loader)) 
    validation_accuracy = 100. * correct / len(test_loader.dataset)
    validation.append(validation_accuracy)
    
    return test_loss, validation_accuracy

In [ ]:
# Checking train, val loss and accuracy

total = []

for epoch in range(1, args.epochs):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, validation_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tValidation Accuracy: {:.2f} % \n".format(
        epoch, test_loss, validation_accuracy))
    
    total.append((test_loss, validation_accuracy))

In [ ]:
# total

In [ ]:
nb_classes = args.num_classes
confusion_matrix = np.zeros((nb_classes, nb_classes))
classes = {
    "0": "Depression",
    "1": "Normal",
    "2": "Suicidality"
}

with torch.no_grad():
    for i, (image, label) in enumerate(test_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        outputs = model(image)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(label.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

plt.figure(figsize=(8,5))
print(confusion_matrix)

class_names = list(classes.values())
df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names).astype(int)
heatmap = sns.heatmap(df_cm, annot=True, fmt="d")

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right',fontsize=8)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right',fontsize=8)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.savefig('dep_train_entire_output.png')

In [ ]:
cm = confusion_matrix
total = sum(sum(cm))

## Accuracy, Sensitivity, and Specificity
acc = (cm[0,0]+cm[1,1]+cm[2,2]) / total
sen_dep = cm[0,0] / (cm[0,0] + cm[0,1] + cm[0,2])
sen_nor = cm[1,1] / (cm[1,0] + cm[1,1] + cm[1,2])
sen_sui = cm[2,2] / (cm[2,0] + cm[2,1] + cm[2,2])

spe_dep = (cm[1,1] + cm[2,2]) / (cm[1,0] + cm[2,0] + cm[1,1] + cm[2,2])
spe_nor = (cm[0,0] + cm[2,2]) / (cm[0,1] + cm[2,1] + cm[0,0] + cm[2,2])
spe_sui = (cm[0,0] + cm[1,1]) / (cm[0,2] + cm[1,2] + cm[0,0] + cm[1,1])

print("Overall classification accuracy is :", round(acc, 4))
print("sensitivity of depression class is :", round(sen_dep, 4))
print("sensitivity of normal class is :", round(sen_nor,4))
print("sensitivity of suicidal class is :", round(sen_sui,4))

print("specificity of depression class is :", round(spe_dep,4))
print("specificity of normal class is :", round(spe_nor,4))
print("specificity of suicidal class is :", round(spe_sui,4))

## Ensemble methods

In [ ]:
from torchensemble import VotingClassifier

In [ ]:
## Defining ensemble
ensemble = VotingClassifier(
    estimator = model,
    n_estimators = 2
)

In [ ]:
## Set the criterion and optimizer
criterion = nn.CrossEntropyLoss()
ensemble.set_criterion(criterion)
ensemble.set_optimizer(
    "Adam", lr = args.lr
)

ensemble.set_scheduler(
    "CosineAnnealingLR", T_max=args.epochs
)

# Train the ensemble
ensemble.fit(
    train_loader, epochs=args.epochs
)

# Evaluate the ensemble
acc = ensemble.predict(test_loader)